# Topic modelling of news headlines for prediction of news category with 4 topics¶

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier on the combined feature 'title_description_text' feature and assume 4 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import pyLDAvis.lda_model

## Load data

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

In [6]:
# append words to stopwords that contain no information
stop_words += ['wegen', 'melden', 'meldet', 'können', 'könnte', 'könnten', 'update', 'neu', 'neue', 'neues', 'ohne', 'letzte', 'letzter',
                  'letztes', 'eins', 'zwei', 'drei', 'vier', 'fünf', 'sechs', 'sieben', 'acht', 'neun', 'zehn',
                   'gehen', 'geht', 'wollen', 'wollte', 'wollt', 'jahr', 'jahre', 'fordern', 'fordert',
                  'warnen', 'warnt', 'frühjahr', 'frühling', 'sommer', 'herbst', 'winter', 'erneut', 'deutlich', 'schwer', 'jahren', 'woche',
                  'wochen', 'monat', 'monate', 'tag', 'tage', 'stunden', 'stunde', 'minuten', 'minuten', 'ende', 'beenden', 'endet']

## Extract features from 'title_description_text' 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer(max_df=0.95, min_df=3, stop_words=stop_words)

In [9]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title_description_text_cleaned'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [10]:
dtm

<67513x29998 sparse matrix of type '<class 'numpy.int64'>'
	with 1026097 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

In [12]:
lda = LatentDirichletAllocation(n_components=4, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=4, random_state=42)

## Analyse extracted features

In [13]:
len(cv.get_feature_names_out())

29998

In [14]:
cv.get_feature_names_out()

array(['000', '01', '034', ..., 'đoković', 'войны', 'украинцы'],
      dtype=object)

In [15]:
len(lda.components_)

4

In [16]:
lda.components_

array([[ 7.60501273,  0.25295922, 31.7886554 , ...,  0.25002535,
         0.25040763,  5.24954511],
       [27.9948475 ,  0.25973101, 13.32410326, ...,  0.25200651,
         0.25044091,  0.25017456],
       [18.12925748,  1.2470038 , 40.22519753, ...,  3.24659232,
         3.2487361 ,  0.25011584],
       [ 7.2708823 ,  3.24030597,  4.6620438 , ...,  0.25137582,
         0.25041536,  0.25016449]])

In [17]:
len(lda.components_[0])

29998

### Show most important words of first extracted topic

In [18]:
first_topic = lda.components_[0]

In [19]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([ 9857, 28495, 28950, ..., 14353, 21156, 25501], dtype=int64)

In [20]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [21]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

moskau
selenskyj
präsident
kiew
ukrainisch
putin
russisch
krieg
russland
ukraine


### Show most important words of first extracted topic

In [22]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['geben', 'liveblog', 'deutschland', 'entwicklung', 'angriff', 'moskau', 'selenskyj', 'präsident', 'kiew', 'ukrainisch', 'putin', 'russisch', 'krieg', 'russland', 'ukraine']


Die TOP-15 Wörter für das Thema #1
['berlin', 'gewinnen', 'erster', 'stehen', 'sterben', 'fußball', 'mensch', 'geben', 'mann', 'deutsch', 'frau', 'bayern', 'polizei', 'wm', 'afd']


Die TOP-15 Wörter für das Thema #2
['online', 'hoch', 'focus', 'cdu', 'grüne', 'scholz', 'groß', 'habeck', 'ampel', 'spd', 'berlin', 'gut', 'geben', 'deutsch', 'deutschland']


Die TOP-15 Wörter für das Thema #3
['türkei', 'iran', 'regierung', 'deutschland', 'geben', 'biden', 'mensch', 'trump', 'land', 'china', 'usa', 'präsident', 'corona', 'eu', 'us']




### Get topics with highest probability for news items in train data

In [23]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 4)

In [24]:
topic_results[0].round(5)

array([0.01108, 0.01117, 0.96649, 0.01126])

In [25]:
topic_results[0].argmax()

2

In [26]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [27]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...,2
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...,0
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...,2
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...,0
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...,3


## Visualize model

In [28]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2     -55.518112  20.807894       1        1  32.652858
0      -3.288924  69.505104       2        1  23.046099
3      -7.352235 -31.665487       3        1  22.891779
1      45.004555  16.861027       4        1  21.409264, topic_info=           Term          Freq         Total Category  logprob  loglift
25501   ukraine  11706.000000  11706.000000  Default  30.0000  30.0000
21156  russland   5941.000000   5941.000000  Default  29.0000  29.0000
14353     krieg   5892.000000   5892.000000  Default  28.0000  28.0000
21150  russisch   5603.000000   5603.000000  Default  27.0000  27.0000
19899     putin   3913.000000   3913.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
10573      groß    670.749907   3645.921240   Topic4  -5.9190  -0.1516
3938    bleiben    561.961642   2185.213329   Topic4  -6.0960   0.1833
7618       euro    544.180798   1945.657113   Topic4  -6.1281   0.2673
29064    zeigen    550.202791   2581.579322   Topic4  -6.1171  -0.0045
11479     heute    505.937381   1814.233483   Topic4  -6.2010   0.2643

[283 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
627        4  0.999581          afd
643        3  0.998769  afghanistan
706        4  0.996263     aiwanger
979        1  0.998681        ampel
979        3  0.000594        ampel
...      ...       ...          ...
28982      4  0.005299         york
29064      1  0.420285       zeigen
29064      2  0.214597       zeigen
29064      3  0.152232       zeigen
29064      4  0.213048       zeigen

[490 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])

## Save train set with topics and dump countvectorizer and model

In [29]:
#Save result to csv
train.to_csv('evaluation/train_lda_combined_4.csv')

In [30]:
# Dump countvectorizer
joblib.dump(cv, 'models/cv_combined.jl')

['models/cv_combined.jl']

In [31]:
# Dump LDA model
joblib.dump(lda, 'models/lda_model_4_topics_combined.jl')

['models/lda_model_4_topics_combined.jl']